In [1]:
#Just import anything I might need
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import os.path 
from os import path
import shutil
import requests
import io

import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [2]:
moviesDF = pd.read_csv("moviesCopy.csv")

moviesDF.head(10)

,Unnamed: 0,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,Profit
0,0,8000000,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414,Stand by Me,R,8/22/1986,89,8.1,Wil Wheaton,299174,Stephen King,1986,44287414
1,1,6000000,Paramount Pictures,USA,John Hughes,Comedy,70136369,Ferris Bueller's Day Off,PG-13,6/11/1986,103,7.8,Matthew Broderick,264740,John Hughes,1986,64136369
2,2,15000000,Paramount Pictures,USA,Tony Scott,Action,179800601,Top Gun,PG,5/16/1986,110,6.9,Tom Cruise,236909,Jim Cash,1986,164800601
3,3,18500000,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248,Aliens,R,7/18/1986,137,8.4,Sigourney Weaver,540152,James Cameron,1986,66660248
4,4,9000000,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613,Flight of the Navigator,PG,8/1/1986,90,6.9,Joey Cramer,36636,Mark H. Baker,1986,9564613
5,5,6000000,Hemdale,UK,Oliver Stone,Drama,138530565,Platoon,R,2/6/1987,120,8.1,Charlie Sheen,317585,Oliver Stone,1986,132530565
6,6,25000000,Henson Associates (HA),UK,Jim Henson,Adventure,12729917,Labyrinth,PG,6/27/1986,101,7.4,David Bowie,102879,Dennis Lee,1986,-12270083
7,7,6000000,De Laurentiis Entertainment Group (DEG),USA,David Lynch,Drama,8551228,Blue Velvet,R,10/23/1986,120,7.8,Isabella Rossellini,146768,David Lynch,1986,2551228
8,8,9000000,Paramount Pictures,USA,Howard Deutch,Comedy,40471663,Pretty in Pink,PG-13,2/28/1986,96,6.8,Molly Ringwald,60565,John Hughes,1986,31471663
9,9,15000000,SLM Production Group,USA,David Cronenberg,Drama,40456565,The Fly,R,8/15/1986,96,7.5,Jeff Goldblum,129698,George Langelaan,1986,25456565


In [3]:
#Copy our splitting code in


#Split code
def fractional_split(data_set, test_fraction=0.5,seed=3410):
    data_count = len(data_set)
    test_count = int(test_fraction*data_count)
    
    np.random.seed(seed)
    shuffled_indices = np.random.permutation(data_count)
    
    #use the front of the shuffled list as test set
    #use the back of the shuffled list as the training set
    test_indices = shuffled_indices(test_count)
    train_indices = shuffled_indices(test_count)
    
    return data_set.iloc[train_indeces], data_set.iloc[test_indeces]

## Target features

x = budget and score
y = profit

I want to predict profit, are there any factors that you can use to maximize your profit? Additionally, based on budget, score, year potentially, runtime, rating I feel like all of those combined could make an excellent predictor because some movies just don't fare well because their audience is weakened due to rating (younger people don't go) or runtime (adults without that much time)

I'm going to start off with budget and score.

In [4]:
#Make decision tree classifier


moviesDF = pd.read_csv("moviesCopy.csv")


#Greatly slim up data because it just won't run
moviesDF = moviesDF.drop(labels="director", axis=1)
moviesDF = moviesDF.drop(labels="released", axis=1)
moviesDF = moviesDF.drop(labels="star", axis=1)
moviesDF = moviesDF.drop(labels="writer", axis=1)
moviesDF = moviesDF.drop(labels="year", axis=1)
moviesDF = moviesDF.drop(labels="company", axis=1)
moviesDF = moviesDF.drop(labels="Unnamed: 0", axis=1)
moviesDF = moviesDF.drop(labels="country", axis=1)
moviesDF = moviesDF.drop(labels="rating", axis=1)
#moviesDF = moviesDF.drop(labels="Unnamed: 0", axis=1) #idk why this even exists
#moviesDF = moviesDF.drop(labels="", axis=1) #Not sure if this one is needed there is an ampty column

display(moviesDF.corr(method="kendall"))

moviesDF.to_csv("moviesCopyTrimmed.csv")


#Just reopen the thing, try to root out any possibility it goes through the old data
moviesDF = pd.read_csv("moviesCopyTrimmed.csv")


#Create training and test set with *reduced* data
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(moviesDF, test_size=0.9,random_state=938)

#print some information
display(len(train_set), len(test_set))
display(train_set.head())
print("\n\n")
display(test_set.head())



#Define x, y, z. Note z is going to be the same concept as x, testing two different models
x = train_set[['budget', 'score']] #Primary testing
z = train_set[['score', 'runtime']] #secondary option
y = train_set['Profit']


#Make linear model
svc_classifier = SVC(kernel='linear')


#Fit data
svc_classifier.fit(x, y)
svc_classifier.fit(z, y)


,budget,gross,runtime,score,votes,Profit
budget,1.000000,0.523359,0.165126,-0.019559,0.451181,-0.134415
gross,0.523359,1.000000,0.138995,0.033544,0.480412,0.379388
runtime,0.165126,0.138995,1.000000,0.281176,0.195930,-0.001658
score,-0.019559,0.033544,0.281176,1.000000,0.273633,0.115024
votes,0.451181,0.480412,0.195930,0.273633,1.000000,0.150601
Profit,-0.134415,0.379388,-0.001658,0.115024,0.150601,1.000000


677

6095

,Unnamed: 0,budget,genre,gross,name,runtime,score,votes,Profit
2881,2881,90000000,Animation,245852179,Toy Story 2,92,7.9,420432,155852179
1672,1672,0,Drama,1610792,Wide Sargasso Sea,98,5.8,1662,1610792
4588,4588,0,Comedy,10205,Taxidermia,91,7.0,14123,10205
28,28,25000000,Action,79817937,The Golden Child,94,5.9,42997,54817937
2930,2930,21000000,Animation,52008288,"South Park: Bigger, Longer & Uncut",81,7.8,171511,31008288


,Unnamed: 0,budget,genre,gross,name,runtime,score,votes,Profit
4476,4476,100000000,Animation,198000317,Happy Feet,108,6.5,144200,98000317
3798,3798,29000000,Action,56359780,Once Upon a Time in Mexico,102,6.4,137276,27359780
2404,2404,19000000,Crime,1353824,Kansas City,116,6.3,3727,-17646176
788,788,2300000,Action,792966,The Toxic Avenger Part II,102,5.1,5067,-1507034
3839,3839,50000000,Action,40905277,Out of Time,105,6.5,49072,-9094723


SVC(kernel='linear')

In [5]:
#Warning, this will take FOREVER. Run on desktop pc NOT laptop.

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier


#Make tree classifier, we are going to test both
tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(x,y)



#classify based on budget and score
print("\n\n\nTree classifier x")
y_predicted = tree_classifier.predict(x) #For the first matrix
print ("Accuracy is ", accuracy_score(y, y_predicted))
print ("Precision is ", precision_score(y, y_predicted, average="weighted"))
print ("Sensitivity is ", recall_score(y, y_predicted, average="weighted"))
print ("F1 is ", f1_score(y, y_predicted, average="weighted"))


#Classify based on score and runtime
print("\n\n\nTree classifier z")
y_predicted = tree_classifier.predict(z) #For the second matrix
print ("Accuracy is ", accuracy_score(y, y_predicted))
print ("Precision is ", precision_score(y, y_predicted, average="weighted"))
print ("Sensitivity is ", recall_score(y, y_predicted, average="weighted"))
print ("F1 is ", f1_score(y, y_predicted, average="weighted"))






Tree classifier x
Accuracy is  0.6499261447562777
Precision is  0.5807935130682546
Sensitivity is  0.6499261447562777
F1 is  0.5958138037901701



Tree classifier z
Accuracy is  0.0014771048744460858
Precision is  2.181838810112387e-06
Sensitivity is  0.0014771048744460858
F1 is  4.357241517540076e-06


c:\users\gabriel\ds-venv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\gabriel\ds-venv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
print("\n\n\nsvc classifier x")
y_predicted = svc_classifier.predict(x)
print ("Accuracy is ", accuracy_score(y, y_predicted))
print ("Precision is ", precision_score(y, y_predicted, average="weighted"))
print ("Sensitivity is ", recall_score(y, y_predicted, average="weighted"))
print ("F1 is ", f1_score(y, y_predicted, average="weighted"))

print("\n\n\nsvc classifier z")
y_predicted = svc_classifier.predict(z)
print ("Accuracy is ", accuracy_score(y, y_predicted))
print ("Precision is ", precision_score(y, y_predicted, average="weighted"))
print ("Sensitivity is ", recall_score(y, y_predicted, average="weighted"))
print ("F1 is ", f1_score(y, y_predicted, average="weighted"))




svc classifier x
Accuracy is  0.0029542097488921715
Precision is  9.49938254013489e-06
Sensitivity is  0.0029542097488921715
F1 is  1.8932952198227284e-05



svc classifier z


c:\users\gabriel\ds-venv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy is  0.8020679468242246
Precision is  0.713121614967996
Sensitivity is  0.8020679468242246
F1 is  0.7413096996553421


c:\users\gabriel\ds-venv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
Same comments as above